Program for finding seeing from star cubes by fitting gaussians to the star in each image.

In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from numpy import pi, r_
from scipy import optimize
import math

In [ ]:
#programs to fit a Gaussian to the stars:
#From http://scipy-cookbook.readthedocs.io/items/FittingData.html

def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape)
    x = (X*data).sum()/total
    y = (Y*data).sum()/total
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum())
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum())
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

def fwhm(width):
    """Takes widths from above programs, returns FWHM"""
    return 2*math.sqrt(2*np.log(2))*width

In [ ]:
# load cal star cubes
# cubes that is closest to Jupiter data - star6

imlist = np.loadtxt('/Volumes/external_hd/march2017/redo/calstars/star6',dtype=str)
directory = '/Volumes/external_hd/march2017/redo/calstars/'

# load coordinates surrounding a star. Closely cropping the star allows the Gaussian fitter to avoid errors.
x1 = 200
x2 = 300
y1 = 200
y2 = 300

wl_list = []
x1_list = []
y1_list = []
fwhm_x_list = []
fwhm_y_list = []

for i in range(0,len(imlist)):
    im = fits.open(directory+imlist[i])
    if im[0].header['rfon'] == 1:
        wl_list.append(im[0].header['orig_wl'])
        gauss = fitgaussian(im[0].data[y1:y2,x1:x2])
        x1, y1, fwhm_x, fwhm_y  = gauss[1], gauss[2], fwhm(gauss[3]), fwhm(gauss[4])
        x1_list.append(200+x1)
        y1_list.append(200+y1)
        fwhm_x_list.append(fwhm_x)
        fwhm_y_list.append(fwhm_y)
        print 'x and y position of star:', y1, x1
        #print gauss[3], gauss[4]
        print 'FWHM x and y values:',fwhm_x, fwhm_y
        print 'Wavelength:',im[0].header['orig_wl']
        plt.matshow(im[0].data[y1:y2,x1:x2])
        plt.plot(y1,x1,'ro')
        plt.show()
    im.close()

In [ ]:
plt.plot(wl_list,np.array(x1_list)+x1)
plt.ylabel('X-position')
plt.xlabel('Wavelength (nm)')
plt.show()

plt.plot(wl_list,np.array(y1_list)+y1)
plt.ylabel('Y-position')
plt.xlabel('Wavelength (nm)')
plt.show()

In [ ]:
plt.plot(wl_list,abs(np.array(fwhm_x_list)))
plt.ylabel('FWHM (x-direction)')
plt.show()

plt.plot(wl_list,abs(np.array(fwhm_y_list)))
plt.ylabel('FWHM (y-direction)')
plt.show()

In [ ]:
# mean X and Y FWHM in pixels for the star cube before Jupiter's
print np.mean(abs(np.array(fwhm_x_list)))
print np.mean(abs(np.array(fwhm_y_list)))

In [ ]:
# find average FWHM size using those values
np.mean([7.56447635299,7.74006015751])

In [ ]:
#plate scale for old camera:
plate_scale = 0.1027 #"/pix

In [ ]:
# Average seeing (arcseconds):
7.6522682552500001*plate_scale

In [ ]:
0.785887949814175 # seeing for Jupiter cube from march 2017